In [75]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, BaggingClassifier, VotingClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.neural_network import MLPClassifier

import numpy as np
import xgboost

### Voting Classifiers

In [18]:
iris = datasets.load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [19]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

In [20]:
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

In [21]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9333333333333333
RandomForestClassifier 0.9333333333333333
SVC 0.9333333333333333
VotingClassifier 0.9333333333333333


### Bagging and Pasting in Scikit-Learn

In [23]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500, max_samples=100, bootstrap=True, n_jobs=-1
)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [24]:
accuracy_score(y_test, y_pred)

0.9333333333333333

### Out-of-Bag Evaluation

In [25]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500, max_samples=100, bootstrap=True, n_jobs=-1, oob_score=True
)

In [26]:
bag_clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1, oob_score=True)

In [27]:
bag_clf.oob_score_

0.975

In [28]:
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9333333333333333

In [29]:
bag_clf.oob_decision_function_

array([[0.        , 0.04090909, 0.95909091],
       [0.        , 0.01492537, 0.98507463],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 0.99528302, 0.00471698],
       [0.        , 1.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.9952381 , 0.0047619 ],
       [0.        , 0.        , 1.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 0.70720721, 0.29279279],
       [0.        , 1.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [1.

### Random Forests

In [30]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1)

In [31]:
y_pred_rf = rnd_clf.predict(X_test)

In [32]:
accuracy_score(y_test, y_pred_rf)

0.9333333333333333

In [33]:
bag_clf = BaggingClassifier(
            DecisionTreeClassifier(splitter='random', max_leaf_nodes=16),
            n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
) # equivalent to rnd_clf

### Feature Importance

In [34]:
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.0966382773899211
sepal width (cm) 0.022806661913676984
petal length (cm) 0.44892043805043375
petal width (cm) 0.4316346226459683


### AdaBoost

$$ r_{j} = \frac{\underset{\widehat{y}_{j}^{(i)} \neq y^{(i)}}{\sum_{i = 1}^{m}}w^{(i)}}{\sum_{i = 1}^{m}w^{(i)}} $$

$$ \alpha_{j} = \eta \log \frac{1 - r_{j}}{r_{j}} $$

$$ \mathrm{for\ } i = \mathrm{1, 2, ..., }\ m $$
$$ w^{(i)} \leftarrow  \left\{\begin{matrix}
w^{(i)}\ \mathrm{if\ } \widehat{y}^{(i)} = y^{(i)}
\\ w^{(i)}\exp (\alpha_{j})\ \mathrm{if\ } \widehat{y}^{(i)} \neq y^{(i)}
\end{matrix}\right.
$$

$$ \widehat{y}(\mathbf{x}) = \mathrm{argmax} \underset{\widehat{y}_{j}(\mathbf{x})\ =\ k}{\sum_{j = 1}^{N}}\alpha_{(j)} $$

In [36]:
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=200, 
                             algorithm='SAMME.R', learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### Gradient Boosting

In [42]:
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X ** 2 + X + 2 + np.random.randn(m, 1)

In [45]:
tree_reg1 = DecisionTreeRegressor()
tree_reg1.fit(X, y)

DecisionTreeRegressor()

In [46]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor()
tree_reg2.fit(X, y2)

DecisionTreeRegressor()

In [47]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor()
tree_reg3.fit(X, y3)

DecisionTreeRegressor()

In [48]:
y_pred = sum(tree.predict(X) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [50]:
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [53]:
X_train, X_val, y_train, y_val = train_test_split(X, y)

In [55]:
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GradientBoostingRegressor(max_depth=2, n_estimators=120)

In [56]:
errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
best_n_estimators = np.argmin(errors) + 1

In [57]:
gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=best_n_estimators)
gbrt_best.fit(X_train, y_train)

/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GradientBoostingRegressor(max_depth=2, n_estimators=73)

In [58]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float('inf')
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        error_going_up = 0
        min_val_error = val_error
    else:
        error_going_up += 1
        if error_going_up == 5:
            break # early stopping

/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

In [62]:
xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [64]:
xgb_reg.fit(X_train, y_train,
           eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:3.27414
[1]	validation_0-rmse:2.52196
[2]	validation_0-rmse:2.02780
[3]	validation_0-rmse:1.68564
[4]	validation_0-rmse:1.43475
[5]	validation_0-rmse:1.28701
[6]	validation_0-rmse:1.17394
[7]	validation_0-rmse:1.11183
[8]	validation_0-rmse:1.06905
[9]	validation_0-rmse:1.06493
[10]	validation_0-rmse:1.06946
[11]	validation_0-rmse:1.05610
[12]	validation_0-rmse:1.04851
[13]	validation_0-rmse:1.05859


### Exercises

#### 8.

In [66]:
mnist = datasets.fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(np.int64)

In [67]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=42)

In [71]:
random_forest_clf = RandomForestClassifier(n_estimators=10, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=10, random_state=42)
svm_clf = LinearSVC(random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [73]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the RandomForestClassifier(n_estimators=10, random_state=42)
Training the ExtraTreesClassifier(n_estimators=10, random_state=42)
Training the LinearSVC(random_state=42)


/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Training the MLPClassifier(random_state=42)


In [74]:
[estimator.score(X_val, y_val) for estimator in estimators]

[0.9469, 0.9492, 0.8695, 0.9663]

In [76]:
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf),
]

In [77]:
voting_clf = VotingClassifier(named_estimators)

In [78]:
voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(n_estimators=10,
                                                     random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(n_estimators=10,
                                                   random_state=42)),
                             ('svm_clf', LinearSVC(random_state=42)),
                             ('mlp_clf', MLPClassifier(random_state=42))])

In [79]:
voting_clf.score(X_val, y_val)

0.9626

In [80]:
[estimator.score(X_val, y_val) for estimator in voting_clf.estimators_]

[0.9469, 0.9492, 0.8695, 0.9663]

In [81]:
voting_clf.set_params(svm_clf=None)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(n_estimators=10,
                                                     random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(n_estimators=10,
                                                   random_state=42)),
                             ('svm_clf', None),
                             ('mlp_clf', MLPClassifier(random_state=42))])

In [82]:
voting_clf.estimators

[('random_forest_clf',
  RandomForestClassifier(n_estimators=10, random_state=42)),
 ('extra_trees_clf', ExtraTreesClassifier(n_estimators=10, random_state=42)),
 ('svm_clf', None),
 ('mlp_clf', MLPClassifier(random_state=42))]

In [83]:
voting_clf.estimators_

[RandomForestClassifier(n_estimators=10, random_state=42),
 ExtraTreesClassifier(n_estimators=10, random_state=42),
 LinearSVC(random_state=42),
 MLPClassifier(random_state=42)]

In [84]:
del voting_clf.estimators_[2]

In [85]:
voting_clf.score(X_val, y_val)

0.9649

In [86]:
voting_clf.voting = "soft"

In [87]:
voting_clf.score(X_val, y_val)

0.9715

In [88]:
voting_clf.score(X_test, y_test)

0.9693

In [89]:
[estimator.score(X_test, y_test) for estimator in voting_clf.estimators_]

[0.9437, 0.9474, 0.9636]

#### 9.

In [90]:
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [91]:
X_val_predictions

array([[5., 5., 5., 5.],
       [8., 8., 8., 8.],
       [2., 2., 2., 2.],
       ...,
       [7., 7., 7., 7.],
       [6., 6., 6., 6.],
       [7., 7., 7., 7.]], dtype=float32)

In [92]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [93]:
rnd_forest_blender.oob_score_

0.9624

In [95]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [98]:
y_pred = rnd_forest_blender.predict(X_test_predictions)

In [99]:
accuracy_score(y_test, y_pred)

0.9617